In [2]:
import openai, os

openai.api_key = os.environ.get("OPENAI_API_KEY")

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", max_tokens=2048, temperature=0.5)
multiply_prompt = PromptTemplate(template="请计算一下{question}是多少?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_prompt, output_key="answer")
answer = math_chain.run({"question": "352乘以493"})
print("OpenAI API 说答案是:", answer)

python_answer = 352 * 493
print("Python 说答案是:", python_answer)


OpenAI API 说答案是: 

352乘以493等于173536。
Python 说答案是: 173536


In [3]:
multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer = math_chain.run({"question": "352乘以493"})
print(answer)




num1 = 352
num2 = 493

# 方法一：使用内置函数
result = num1 * num2

# 方法二：使用循环累加
result = 0
for i in range(num2):
    result += num1

print("352乘以493的结果为：", result)


In [6]:
multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer_code = math_chain.run({"question": "352乘以493"})

from langchain_experimental.utilities.python import PythonREPL

python_repl = PythonREPL()
result = python_repl.run(answer_code)
print(result)


Python REPL can execute arbitrary code. Use with caution.


173536



In [7]:
from langchain import LLMMathChain

llm_math = LLMMathChain(llm=llm, verbose=True)
result = llm_math.run("请计算一下352乘以493是多少?")
print(result)


/var/folders/7l/brdxy3kj6c9g9xvg5k_0kw3h0000gn/T/ipykernel_51602/3695818906.py:3: LangChainDeprecationWarning: This class is deprecated and will be removed in langchain 1.0. See API reference for replacement: https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html
  llm_math = LLMMathChain(llm=llm, verbose=True)
/Users/zouguoyang/opt/anaconda3/envs/langchain/lib/python3.9/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
请计算一下352乘以493是多少?```text
352 * 493
```
...numexpr.evaluate("352 * 493")...

Answer: 173536
> Finished chain.
Answer: 173536


In [12]:
from langchain.chains import LLMRequestsChain

template = """在 >>> 和 <<< 直接是来自Google的原始搜索结果.
请把对于问题 '{query}' 的答案从里面提取出来，如果里面没有相关信息的话就说 "找不到"
请使用如下格式：
Extracted:<answer or "找不到">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)
requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT))
question = "现在是北京时间多少，上海的天气怎么样？"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+")
}
result=requests_chain(inputs)
print(result)
print(result['output'])


{'query': '现在是北京时间多少，上海的天气怎么样？', 'url': 'https://www.google.com/search?q=现在是北京时间多少，上海的天气怎么样？', 'output': '  现在北京时间是2024年10月10日13:54:25，上海的天气为11~19°。'}
  现在北京时间是2024年10月10日13:54:25，上海的天气为11~19°。


通过 VectorDBQA 来实现先搜索再回复的能力

In [15]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import SpacyTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import TextLoader

llm = OpenAI(temperature=0)
loader = TextLoader('./data/demo.txt')
documents = loader.load()
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="zh_core_web_sm")
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(texts, embeddings)

faq_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=docsearch, verbose=True)


/var/folders/7l/brdxy3kj6c9g9xvg5k_0kw3h0000gn/T/ipykernel_51602/2211358665.py:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
/Users/zouguoyang/opt/anaconda3/envs/langchain/lib/python3.9/site-packages/langchain/chains/retrieval_qa/base.py:316: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [16]:
texts

[Document(metadata={'source': './data/demo.txt'}, page_content='望着测验魔石碑上面闪亮得甚至有些刺眼的五个大字，少年面无表情，唇角有着一抹自嘲，紧握的手掌，因为大力，而导致略微尖锐的指甲深深的刺进了掌心之中，带来一阵阵钻心的疼痛…\n\n萧炎，斗之力，三段！\n\n级别：低级！”\n\n测验魔石碑之旁，一位中年男子，看了一眼碑上所显示出来的信息，语气漠然的将之公布了出来…\n\n中年男子话刚刚脱口，便是不出意外的在人头汹涌的广场上带起了一阵嘲讽的骚动。\n\n“三段？\n\n嘿嘿，果然不出我所料，这个“天才”这一年又是在原地踏步！”\n\n“哎，这废物真是把家族的脸都给丢光了。”'),
 Document(metadata={'source': './data/demo.txt'}, page_content='萧炎，斗之力，三段！\n\n级别：低级！”\n\n测验魔石碑之旁，一位中年男子，看了一眼碑上所显示出来的信息，语气漠然的将之公布了出来…\n\n中年男子话刚刚脱口，便是不出意外的在人头汹涌的广场上带起了一阵嘲讽的骚动。\n\n“三段？\n\n嘿嘿，果然不出我所料，这个“天才”这一年又是在原地踏步！”\n\n“哎，这废物真是把家族的脸都给丢光了。”\n\n“要不是族长是他的父亲，这种废物，早就被驱赶出家族，任其自生自灭了，哪还有机会待在家族中白吃白喝。”\n\n“唉，昔年那名闻乌坦城的天才少年，如今怎么落魄成这般模样了啊？”'),
 Document(metadata={'source': './data/demo.txt'}, page_content='中年男子话刚刚脱口，便是不出意外的在人头汹涌的广场上带起了一阵嘲讽的骚动。\n\n“三段？\n\n嘿嘿，果然不出我所料，这个“天才”这一年又是在原地踏步！”\n\n“哎，这废物真是把家族的脸都给丢光了。”\n\n“要不是族长是他的父亲，这种废物，早就被驱赶出家族，任其自生自灭了，哪还有机会待在家族中白吃白喝。”\n\n“唉，昔年那名闻乌坦城的天才少年，如今怎么落魄成这般模样了啊？”\n\n“谁知道呢，或许做了什么亏心事，惹得神灵降怒了吧…”'),
 Document(metadata={'source': './data/demo.txt'},

In [33]:
question = "萧炎是最厉害的吗，请用中文回答"
result = faq_chain.run(question)
print(result)




> Entering new VectorDBQA chain...

> Finished chain.
 不，萧炎不是最厉害的。


Langflow是啥

Langflow is an open-source UI-based tool for building and deploying language model-powered applications, specifically designed to work with frameworks like LangChain. It allows users to visually design complex workflows by connecting various components such as language models (like GPT), chains, and tools (like databases or APIs), making it easier to integrate and manage these models in different projects.

With Langflow, you can drag and drop components to create custom pipelines, manage configurations, and easily test workflows without needing to write much code. It’s particularly useful for developers working with large language models and natural language processing tasks who want a more user-friendly and visual approach to building applications.